In [1]:
from google.colab import drive
import tensorflow as tf
import numpy as np
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from IPython.display import Audio, display, Markdown

# Mount Google Drive
drive.mount('/content/drive')
# Load the model
model = tf.keras.models.load_model('/content/drive/MyDrive/Guitar_Chords_Project/saved_models/my_cnn_model.keras')
print("Model loaded successfully!")

Mounted at /content/drive
Model loaded successfully!


In [2]:
from google.colab import files
uploaded = files.upload()
!pip install kaggle
import os
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d fabianavinci/guitar-chords-v2
!unzip guitar-chords-v2.zip -d guitar_chords_data

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/fabianavinci/guitar-chords-v2
License(s): unknown
100% 727M/729M [00:39<00:00, 20.6MB/s]
100% 729M/729M [00:39<00:00, 19.6MB/s]
Archive:  guitar-chords-v2.zip
  inflating: guitar_chords_data/Test/Am/Am_AcousticGuitar_RodrigoMercador_1.wav  
  inflating: guitar_chords_data/Test/Am/Am_AcousticGuitar_RodrigoMercador_2.wav  
  inflating: guitar_chords_data/Test/Am/Am_AcousticGuitar_RodrigoMercador_3.wav  
  inflating: guitar_chords_data/Test/Am/Am_AcousticGuitar_RodrigoMercador_4.wav  
  inflating: guitar_chords_data/Test/Am/Am_AcusticVince_JO_1.wav  
  inflating: guitar_chords_data/Test/Am/Am_AcusticVince_JO_2.wav  
  inflating: guitar_chords_data/Test/Am/Am_AcusticVince_JO_3.wav  
  inflating: guitar_chords_data/Test/Am/Am_AcusticVince_JO_4.wav  
  inflating: guitar_chords_data/Test/Am/Am_Classic2_Carolina_1.wav  
  inflating: guitar_chords_data/Test/Am/Am_Classic2_Carolina_2.wav  
  inflating: guitar_chords_d

In [3]:
def audio_to_spectrogram(file_path, save_path):
    y, sr = librosa.load(file_path, sr=22050)
    spect = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    log_spect = librosa.power_to_db(spect, ref=np.max)

    plt.figure(figsize=(2.5, 2.5))
    librosa.display.specshow(log_spect, sr=sr, x_axis='time', y_axis='mel', fmax=8000)
    plt.axis('off')
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Paths to the audio dataset
training_path = '/content/guitar_chords_data/Training'
test_path = '/content/guitar_chords_data/Test'

# Regenerate spectrograms
for root, dirs, files in os.walk(training_path):
    for file in files:
        if file.endswith('.wav'):
            input_path = os.path.join(root, file)
            output_path = input_path.replace('.wav', '.png').replace('guitar_chords_data', 'spectrogram_data')
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            audio_to_spectrogram(input_path, output_path)

for root, dirs, files in os.walk(test_path):
    for file in files:
        if file.endswith('.wav'):
            input_path = os.path.join(root, file)
            output_path = input_path.replace('.wav', '.png').replace('guitar_chords_data', 'spectrogram_data')
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            audio_to_spectrogram(input_path, output_path)


In [4]:
def preprocess_audio_to_spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    spect = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    log_spect = librosa.power_to_db(spect, ref=np.max)

    # Resize spectrogram to match input size (224x224)
    plt.figure(figsize=(2.5, 2.5))
    librosa.display.specshow(log_spect, sr=sr, x_axis='time', y_axis='mel', fmax=8000)
    plt.axis('off')

    # Save the spectrogram as an image
    spect_image_path = 'temp_spectrogram.png'
    plt.savefig(spect_image_path, bbox_inches='tight', pad_inches=0)
    plt.close()

    # Load the saved image and preprocess it for the model
    img = load_img(spect_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match training data

    # Remove temporary file
    os.remove(spect_image_path)

    return img_array

# Audio file to predict
audio_file_path = '/content/guitar_chords_data/Test/F/F_Electric1_LInda_2.wav'
processed_audio = preprocess_audio_to_spectrogram(audio_file_path)

# Make a prediction
prediction = model.predict(processed_audio)
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Map the predicted index to the corresponding class name
class_labels_path = '/content/spectrogram_data/Training'
class_labels = sorted(os.listdir(class_labels_path))  # Ensure consistent label order

# Display audio player
display(Audio(audio_file_path, autoplay=False))

# Display the predicted class
predicted_class = class_labels[predicted_class_index]
display(Markdown(f"**Predicted Guitar Chord:** `{predicted_class}`"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


**Predicted Guitar Chord:** `F`

# Gradio

In [5]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [8]:
import gradio as gr
import tensorflow as tf
import numpy as np
import librosa
import librosa.display
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Guitar_Chords_Project/saved_models/my_cnn_model.keras')

# Preprocess audio file to spectrogram and prepare it for prediction
def preprocess_audio_to_spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    spect = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    log_spect = librosa.power_to_db(spect, ref=np.max)

    # Resize spectrogram to match input size (224x224)
    plt.figure(figsize=(2.5, 2.5))
    librosa.display.specshow(log_spect, sr=sr, x_axis='time', y_axis='mel', fmax=8000)
    plt.axis('off')

    # Save the spectrogram as an image
    spect_image_path = 'temp_spectrogram.png'
    plt.savefig(spect_image_path, bbox_inches='tight', pad_inches=0)
    plt.close()

    # Load the saved image and preprocess it for the model
    img = load_img(spect_image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match training data

    # Remove temporary file
    os.remove(spect_image_path)

    return img_array

# Prediction function
def predict_guitar_chord(audio_file):
    processed_audio = preprocess_audio_to_spectrogram(audio_file)
    prediction = model.predict(processed_audio)
    predicted_class_index = np.argmax(prediction, axis=1)[0]

    # Map the predicted index to the corresponding class name
    class_labels_path = '/content/spectrogram_data/Training'
    class_labels = sorted(os.listdir(class_labels_path))  # Ensure consistent label order
    predicted_class = class_labels[predicted_class_index]

    return predicted_class

# Gradio Interface
def classify_audio(audio_file):
    prediction = predict_guitar_chord(audio_file)
    return f"Predicted Guitar Chord: {prediction}"

interface = gr.Interface(
    fn=classify_audio,
    inputs=gr.Audio(type="filepath", label="Upload Guitar Audio (.wav)"),
    outputs=gr.Textbox(label="Guitar Chord Prediction"),
    title="Guitar Chord Classifier",
    description="Upload a .wav file of a guitar chord, and the model will predict the chord class."
)

# Launch the app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ee341fbca8750e7144.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
